In [2]:
import os
class Job():
    def __init__(self,START=0,END=60,MODEL="ISING1",load=False,**kwargs):
        """Start and end (epoch), the model, and load (if it loads a previos ednn model) must be specified"""
        self.START=START
        self.END=END
        self.MODEL=MODEL
        self.load="-l" if load else ""
        self.kwargs=kwargs
    def create_file(self,file_name="RNNew.sh"):
        """Creates bash script to run the ednn with the parameters specified on initialization"""
        self.fn = file_name
        fil = open(self.fn,"w")
        fil.write("#!/bin/bash\n#SBATCH --time=24:00:00\n#SBATCH --partition=bumblebee\n#SBATCH --gres=gpu:1\n")
        fil.write("source activate tensorflow\nmodule load cudnn/7.0-9.0\n")
        fil.write("python EDNN.py %s %s %s %s"%(self.MODEL,self.START,self.END,self.load)+ self.__kstring())
        fil.close()
    def __kstring(self):
        """Turns a dictionary into a command line argument strings"""
        return "".join([" %s=%s"%(a,self.kwargs[a]) for a in self.kwargs])
    def run(self,dependency=None,dtype="afterok"):
        if dependency==None:
            a = os.popen("sbatch %s"%self.fn).read()
        else:
            a = os.popen("sbatch --dependency="+dtype+":%s %s"%(dependency.id,self.fn)).read()
        self.id = a.split(" ")[-1].strip()
        assert self.id.isdigit(), a
        return self.id

In [3]:
def jrange(n,m,jo=None):
    """Breaks up an ednn job into gropus of 15 epochs"""
    num=int(n/15+0.94)
    for a in range(num):
        s,e = a*15,a*15+15 if (a+1)*15<n else n
        jc = Job(s,e,MODEL=m,load=a!=0)
        jc.create_file("jobs/%s.sh"%(m.lower()[:-1]+str(a)))
        print(jc.run(jo))
        jo = jc
    return jc
#j = jrange(60,"ISING1")
jo=Job()
jo.id="82667"
j2 = jrange(30,"POTTS1",jo=jo)

82692
82693


In [13]:
14/15


0.9333333333333333